# Create 1MB of query that contains 500 local matches

## Read in local matches and 1MB query sequence

In [54]:
import sys
import os
from Bio import SeqIO
import pandas as pd

def check_file_exists(fn):
    if not os.path.exists(fn):
        raise SystemError("Error: File does not exist\n")

In [55]:
# local matches sampled with certain error rate
match_file = "local_matches_0.fastq"
#match_file = "local_matches_0025.fastq"
#match_file = "local_matches_005.fastq"
#match_file = "local_matches_0075.fastq"
#match_file = "local_matches_010.fastq"


check_file_exists(match_file)


local_matches = list(SeqIO.parse(match_file, "fastq"))
local_matches[0].seq

Seq('GCGGGTCCGTATTCCCAATGCAGCGATTGGAACCTATCTCAGCCTAACAC', SingleLetterAlphabet())

In [56]:
# 1MB of sequence where local matches will be inserted into
query_file = "one_line.fasta"

check_file_exists(query_file)

query = list(SeqIO.parse(query_file, "fasta"))
str(query[0].id)

'1'

## Insert local matches into query at random positions

In [58]:
query_len = len(query[0].seq)
query_len

1048576

In [61]:
import random
ran_ind_list = random.sample(range(query_len), len(local_matches))

In [63]:
ran_ind_list

[870753,
 428325,
 583814,
 87254,
 275930,
 329571,
 723598,
 1037922,
 30745,
 362771,
 556074,
 259714,
 829103,
 861192,
 247361,
 18583,
 398086,
 948291,
 442986,
 930591,
 1037879,
 578701,
 335979,
 979292,
 431507,
 602827,
 460733,
 340438,
 483903,
 129425,
 133641,
 451533,
 706622,
 507919,
 198531,
 699168,
 123121,
 347987,
 147216,
 718346,
 737013,
 1018884,
 556730,
 540658,
 311693,
 889769,
 155981,
 267797,
 944231,
 646066,
 739020,
 985226,
 744057,
 379692,
 279413,
 494076,
 975639,
 852851,
 878599,
 182110,
 1032920,
 979518,
 1030397,
 47807,
 373767,
 293177,
 446337,
 614921,
 994080,
 341556,
 187239,
 636763,
 644060,
 993875,
 234891,
 901714,
 109426,
 1024,
 600672,
 489800,
 594947,
 614772,
 2015,
 281963,
 966174,
 321384,
 697222,
 415469,
 872060,
 376046,
 225917,
 73667,
 612356,
 83318,
 953184,
 26658,
 538235,
 208220,
 439945,
 589215,
 795607,
 845844,
 981860,
 40959,
 106692,
 662990,
 1048323,
 751146,
 590492,
 258489,
 846613,
 27282,

In [71]:
local_matches[0].description = 10
local_matches[0].description

10

In [83]:
for i in range(len(local_matches)):
    local_matches[i].description = ran_ind_list[i]
    
local_matches

[SeqRecord(seq=Seq('GCGGGTCCGTATTCCCAATGCAGCGATTGGAACCTATCTCAGCCTAACAC', SingleLetterAlphabet()), id='l50-0', name='l50-0', description=870753, dbxrefs=[]),
 SeqRecord(seq=Seq('GCATGTCGAAGGTCGTAGTCAACAGGACCGGGAGATGCGCTTGCACGAGC', SingleLetterAlphabet()), id='l50-1', name='l50-1', description=428325, dbxrefs=[]),
 SeqRecord(seq=Seq('ATTACCTCATTGACTCGGAAAACTAACGAGGAAGTGAGCTCTCGATGCTG', SingleLetterAlphabet()), id='l50-2', name='l50-2', description=583814, dbxrefs=[]),
 SeqRecord(seq=Seq('CCGATCCTCAAGAGCGCAGATAGGGTCTACAGGCGTTACCACATTTTGAT', SingleLetterAlphabet()), id='l50-3', name='l50-3', description=87254, dbxrefs=[]),
 SeqRecord(seq=Seq('CCCACCGAGATATTACGCTACCCTCAGGCCGACCCTCATGATTTTCATAC', SingleLetterAlphabet()), id='l50-4', name='l50-4', description=275930, dbxrefs=[]),
 SeqRecord(seq=Seq('TTGATACGATGTTGGGTATGTCGGAAAACGGGGACGAAGTGAGATGTGTA', SingleLetterAlphabet()), id='l50-5', name='l50-5', description=329571, dbxrefs=[]),
 SeqRecord(seq=Seq('AACGGAAAGCGACAGGGATGATCTATGTCTAAGTTACGC

In [85]:
local_matches[0].description

870753

In [88]:
local_matches[0]

SeqRecord(seq=Seq('GCGGGTCCGTATTCCCAATGCAGCGATTGGAACCTATCTCAGCCTAACAC', SingleLetterAlphabet()), id='l50-0', name='l50-0', description=870753, dbxrefs=[])

In [ ]:
#TODO: sort the sequence records based on description (i.e the index)
#      insert sequence records into query based on index